In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
data = pd.read_csv('Sentiment.csv', encoding='ISO-8859-1', header=None)
data.columns = ['polarity', 'id', 'date', 'query', 'user', 'text']
data = data[['polarity', 'text']]
data['polarity'] = data['polarity'].replace({0: 'negative', 4: 'positive'})

data.head()

,polarity,text
0,negative,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,negative,is upset that he can't update his Facebook by ...
2,negative,@Kenichan I dived many times for the ball. Man...
3,negative,my whole body feels itchy and like its on fire
4,negative,"@nationwideclass no, it's not behaving at all...."


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Manis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Manis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data['tokens'] = data['text'].apply(lambda x: word_tokenize(x.lower()))
data['tokens'] = data['tokens'].apply(lambda x: [word for word in x if word not in stop_words])
data['tokens'] = data['tokens'].apply(lambda x: [stemmer.stem(word) for word in x])
data['cleaned_text'] = data['tokens'].apply(lambda x: ' '.join(x))

data[['text', 'cleaned_text']].head()

,text,cleaned_text
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",@ switchfoot http : //twitpic.com/2y1zl - awww...
1,is upset that he can't update his Facebook by ...,upset ca n't updat facebook text ... might cri...
2,@Kenichan I dived many times for the ball. Man...,@ kenichan dive mani time ball . manag save 50...
3,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,"@nationwideclass no, it's not behaving at all....","@ nationwideclass , 's behav . 'm mad . ? ca n..."


In [ ]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(data['cleaned_text'])
y = data['polarity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.78


In [ ]:
def predict_sentiment(text):
    tokens = word_tokenize(text.lower())
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]
    text_vector = tfidf.transform([' '.join(tokens)])
    sentiment = model.predict(text_vector)
    return sentiment[0]

In [ ]:
def chatbot_response(user_input):
    sentiment = predict_sentiment(user_input)
    if sentiment == 'positive':
        return "I'm glad you're feeling positive!"
    elif sentiment == 'negative':
        return "I'm sorry you're feeling down. How can I help?"
    else:
        return "Thanks for sharing!"

In [ ]:
def test_chatbot(custom_inputs):
    for input_text in custom_inputs:
        response = chatbot_response(input_text)
        print(f"User: {input_text}\nChatbot: {response}\n")

custom_inputs = [
    "I absolutely love this new feature!",
    "This is the worst experience I've ever had.",
    "I don't have strong feelings about this.",
    "I'm so excited about the upcoming events!",
    "I feel really down today.",
    "It's okay, nothing special."
]

test_chatbot(custom_inputs)

User: I absolutely love this new feature!
Chatbot: I'm glad you're feeling positive!

User: This is the worst experience I've ever had.
Chatbot: I'm sorry you're feeling down. How can I help?

User: I don't have strong feelings about this.
Chatbot: I'm sorry you're feeling down. How can I help?

User: I'm so excited about the upcoming events!
Chatbot: I'm glad you're feeling positive!

User: I feel really down today.
Chatbot: I'm sorry you're feeling down. How can I help?

User: It's okay, nothing special.
Chatbot: I'm glad you're feeling positive!

